In [2]:
%matplotlib inline
import csv
import networkx as nx
from networkx.algorithms import bipartite
from collections import defaultdict
import matplotlib.pyplot as plt

In [3]:
DATA_DIR = "./data/"
CASES_2010_2014_CSV = "CRM Cases 2010_2014 Unrestricted.csv"
CASES_2015_2019_CSV = "CRM Cases 2015_2019 Unrestricted.csv"
ECOMETRICS_VERTICAL_CSV = "CRM CBG Ecometrics Vertical.csv"
CASE_TYPES_CSV = "CRM Case Types.csv"
SHAPEFILE = "./Block+Groups+2010+BARI/Census Block Groups"

In [8]:
weights = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for filename in [CASES_2015_2019_CSV, CASES_2010_2014_CSV]:
    with open(DATA_DIR + filename, 'r') as f:
        c = csv.DictReader(f, delimiter='\t')
        count = 0
        for row in c:
            count += 1
            open_dt = row.get('OPEN_DT')
            close_dt = row.get('CLOSED_DT')
            reason = row.get('TYPE')
            bg = row.get('BG_ID_10')
            if "NA" in (reason, bg):
                continue
            weights[open_dt[:4]][reason.lower()][bg] += 1
        

In [13]:
Gs = defaultdict(lambda: nx.Graph())
for year in weights:
    for reason in weights[year]:
        Gs[year].add_node(reason, bipartite='reason')
        for bg in weights[year][reason]:
            Gs[year].add_node(bg, bipartite='bg')
            Gs[year].add_edge(reason, bg, weight=weights[year][reason][bg])
Gs

defaultdict(<function __main__.<lambda>>,
            {'2010': <networkx.classes.graph.Graph at 0x86bb410>,
             '2011': <networkx.classes.graph.Graph at 0x867c410>,
             '2012': <networkx.classes.graph.Graph at 0x86bb3b0>,
             '2013': <networkx.classes.graph.Graph at 0x86bb4d0>,
             '2014': <networkx.classes.graph.Graph at 0x8686b90>,
             '2015': <networkx.classes.graph.Graph at 0xaeacc10>,
             '2016': <networkx.classes.graph.Graph at 0x8686b70>})

In [10]:
def export_graph(G, file):
    nx.write_weighted_edgelist(G, file, delimiter=";")
        
#export_graph(bipartite.weighted_projected_graph(G, reason_nodes), "test.txt")

In [13]:
def draw_bipartite(G):
    X, Y = bipartite.sets(G)
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) 
    nx.draw(G, pos=pos)
    plt.show()


In [1]:
def setup_map(shapefile, name, scale):
    from mpl_toolkits.basemap import Basemap
    plt.figure(figsize=(12*scale,6*scale))
    m = Basemap(projection='merc', llcrnrlat=42.22, urcrnrlat=42.42,
                   llcrnrlon=-71.2, urcrnrlon=-70.98, resolution='h')
    m.readshapefile(shapefile, name, linewidth=0.7)
    m.drawcoastlines()
    m.drawstates()
    m.drawmapboundary(fill_color='aqua')
    m.fillcontinents(color='coral',lake_color='aqua')
    return m
    

#m = setup_map(SHAPEFILE, "CBGs", 2)
#try:
   # m.plot()
#except Exception as e:
    #pass

